In [1]:
import os 
import glob
import pandas as pd 
import numpy as np
from main.tree.linkageTree import linkageCut
from main.tsp.TSP_Formulation_Methods import ( 
    create_QUBO_matrix,
    solve_qubo_with_Dwave,
    compute_general_lambdas,
)
from main.tree.utils import ( 
    view_linkage_on_map, 
    draw_centers_on_map,
    map_draw_line,
    convert_bitstring_to_matrix,
    assemble_line,
)
from main.pipe import give_line
from data.utils import fetch_amenities_from

In [2]:
# Load previously stored overpy lat/lon datafile for different amenities
filepath = os.path.join('data', 'amenities-granada.csv')
if os.path.exists(filepath):
    amenities_data = pd.read_csv(filepath)
else:
    # If there is no previous data:
    query_file = os.path.join('data', 'overpy-granada-query.txt')
    query = None
    with open(query_file) as file:
        query = file.read()
    amenities_data = fetch_amenities_from(query=query) # Defaults to Granada
    amenities_data.to_csv(filepath)
    
# Create a hierarchical clustering of amenities
hierarchical_cluster = linkageCut(amenities_data)
# Set a specific number of clusters per levels. Max 9 in this POC
nclusters = 6
levels = 2
labels = hierarchical_cluster.top_down_view_recur(nclusters=nclusters, levels=2)
# Visualize for debugging purposes.
view_linkage_on_map(linkage_matrix=hierarchical_cluster)

In [3]:
# Fetch the centers of the first level
centers = hierarchical_cluster.give_centers_level(0)
# Sanity check by drawing the graph
draw_centers_on_map(centers, color='red')

In [4]:
np.random.seed(42)
# Fetch the distance from the centers of the first level
distances = hierarchical_cluster.dist_matrix_level(0, return_labels=False)
# Set initial global parameters
N = distances.shape[0]
p = 3
node_options = set(np.arange(nclusters) + 1)
startNode_0 = np.random.choice(list(node_options))
endNode_0 = np.random.choice(list(node_options - set([startNode_0])))
print("Selected random nodes:", startNode_0, endNode_0)

# Process Parameters
p = min(p, N-1)
startNode_0 = min(startNode_0, N)
endNode_0 = min(endNode_0, N)

reduced_distances = distances/np.max(distances)
maxDistance = np.max(reduced_distances)
lambdas = compute_general_lambdas(reduced_distances, max_N=3)

Selected random nodes: 4 6


In [5]:
Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode_0 - 1, endNode_0 - 1, lambdas)   

level0_sols, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)

adjacency = convert_bitstring_to_matrix(level0_sols, N=N, p=p)
map_draw_line(centers[:, ::-1], adjacency, color='red')

In [6]:
# Formulation with initial lambdas
level1_sols = {} # Dict that will hold the bitstring, connected level-0 clusters and corresponding start-end nodes 
nchecks = 1024
all_indices = set(np.arange(nclusters - 1) + 1)
for i in range(1, nclusters+1):
    
    connections = np.concatenate([adjacency[:,i-1].nonzero()[0], adjacency[i-1, :].nonzero()[0]], axis=0) + 1
    print("----- Solving level-1:", i, "------\n")
    print("connections", connections)
    if len(connections) > 0: #Selected
        # Fetch the centers of the first level
        distances, closest, _ = hierarchical_cluster.dist_matrix_label_down(
        i,
        connections=connections,
        )
        
        startNode_ = None
        if len(closest) >= 1:
            startNode_ = closest[0]
            choices = all_indices - set([startNode_])
        if len(closest) == 2:
            endNode_ = closest[1]
        else:
            endNode_ = np.random.choice(list(choices)) # POC criterion, better heuristic should be chosen
        print("Start-end", startNode_, endNode_)
        reduced_distances = distances / np.max(distances)
        Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode_ - 1, endNode_ - 1, lambdas)
        


        sol_, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
        level1_sols[i] = [sol_, closest]
        
    else:
        level1_sols[i] = (np.zeros((nclusters*(p + 1))), [])
        print('The line does not cross this level-0 cluster')


----- Solving level-1: 1 ------

connections [5 6]
Start-end 6 1
----- Solving level-1: 2 ------

connections []
The line does not cross this level-0 cluster
----- Solving level-1: 3 ------

connections []
The line does not cross this level-0 cluster
----- Solving level-1: 4 ------

connections [5]
Start-end 3 5
----- Solving level-1: 5 ------

connections [4 1]
Start-end 6 5
----- Solving level-1: 6 ------

connections [1]
Start-end 5 3


In [7]:
example_indx = 5
example_centers = hierarchical_cluster.give_centers_label_down(example_indx)
example_adj = convert_bitstring_to_matrix(level1_sols[example_indx][0],N=nclusters, p=p)
map = map_draw_line(centers[:, ::-1], adjacency, color='red')
map_draw_line(example_centers[:, ::-1], example_adj, color='blue', map=map)

In [8]:
assembled = assemble_line(level0_sols,level1_sols, nclusters, p)
centers_level1 = hierarchical_cluster.give_centers_level(1)
map = map_draw_line(centers_level1[:,::-1], assembled, color='blue', zoom_start=14)
map

[4 5 1 6]


In [9]:
new_lines = 1
assembled_lines = []
assembled_lines.append(assembled)
map = map_draw_line(centers_level1[:,::-1], assembled, color='blue', zoom_start=14, map=map)
colors = ['red', 'green', 'pink', 'orange', 'purple', 'yellow', 'black']

set_of_used_start_nodes = set([startNode_0])
set_of_used_end_nodes = set([endNode_0])

for l in range(new_lines):
    new_nodes = set(np.arange(nclusters) + 1) - set_of_used_start_nodes
    new_startNode_0 = np.random.choice(list(new_nodes))
    new_endNode_0 = np.random.choice(list(new_nodes - set([new_startNode_0])))

    set_of_used_start_nodes.add(new_startNode_0)
    new_line = give_line(amenities_data, nclusters, p, new_startNode_0, new_endNode_0,'qutip', classical=True)

    new_assembled = assemble_line(new_line[0], new_line[1], nclusters, p)
    centers_level1 = hierarchical_cluster.give_centers_level(1)


    map = map_draw_line(centers_level1[:,::-1], new_assembled, color=colors[l], zoom_start=14, map=map)

map

----- Solving level-1: 1 ------

----- Solving level-1: 2 ------

----- Solving level-1: 3 ------

----- Solving level-1: 4 ------

----- Solving level-1: 5 ------

----- Solving level-1: 6 ------

[3 2 1 5]
